<a href="https://colab.research.google.com/github/WilsonForeroB/ColabIA/blob/main/WebSearchRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**RAG Multi-Agente: Síntesis de Fuentes Web con Ollama, LangChain y Gradio**

Objetivo: Construir paso a paso un sistema multi-agente que consulta dos fuentes de internet definidas por el usuario, sintetiza la información y expone una interfaz conversacional accesible desde cualquier navegador.


**¿Qué vamos a construir?**


In [ ]:
Usuario ──► Gradio (UI) ──► Agente Maestro (LangChain)
                                  │
                    ┌─────────────┴─────────────┐
                    ▼                           ▼
             Herramienta A              Herramienta B
           (DuckDuckGo → site:A)   (DuckDuckGo → site:B)
                    │                           │
                    └─────────────┬─────────────┘
                                  ▼
                          Síntesis Final (LLM)
                          + Memoria de sesión

Stack tecnológico:


*   Ollama → Ejecuta el LLM (llama3.2:1b) localmente en Colab
*   LangChain → Orquesta el agente y sus herramientas
*   DuckDuckGo Search → Busca en las URLs configuradas
*   Gradio → Interfaz web interactiva
*   Ngrok → Túnel público para compartir la demo

**CELDA 1 — Instalación de dependencias**

¿Qué hace esta celda?
Instala Ollama (el motor de LLM local) y todas las librerías de Python necesarias. Es el único paso que requiere conexión a internet y puede tardar 2-3 minutos.

⚠️ Ejecútala una sola vez por sesión de Colab.

In [3]:
# ==============================================================================
# CELDA 1: INSTALACIÓN
# ==============================================================================

print("📦 Instalando zstd...")
!apt-get install -y -q zstd

print("📦 Instalando Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

print("📦 Instalando librerías adicionales...")
!pip install -q ddgs duckduckgo-search pyngrok gradio \
                langchain-core \
                langchain-community \
                langchain-ollama

print("\n✅ Listo. Ve a Entorno de ejecución → Reiniciar sesión y luego ejecuta la Celda 2.")

📦 Instalando zstd...
Reading package lists...
Building dependency tree...
Reading state information...
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
📦 Instalando Ollama...
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
📦 Instalando librerías adicionales...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📋 **CELDA 2 — Importaciones y arranque del servidor Ollama**

¿Qué hace esta celda?
Importa todas las clases que usaremos a lo largo del notebook y lanza el servidor de Ollama en segundo plano. El time.sleep(10) le da tiempo al servidor para inicializarse antes de recibir peticiones.

In [4]:
# ==============================================================================
# CELDA 2: IMPORTACIONES Y ARRANQUE DE OLLAMA
# ==============================================================================

import subprocess, time, json
import gradio as gr

# langchain_core: estable en cualquier versión 1.x
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_core.tools import tool

# langchain_community: búsqueda web
from langchain_community.tools import DuckDuckGoSearchRun

# langchain_ollama: interfaz con Ollama
from langchain_ollama import ChatOllama

# Utilidades
from pyngrok import ngrok
from google.colab import userdata

# Arrancar servidor Ollama
print("🚀 Iniciando servidor Ollama...")
subprocess.Popen(["ollama", "serve"])
time.sleep(10)

print("⬇️  Descargando modelo llama3.2:1b ...")
!ollama pull llama3.2:1b

print("✅ Servidor y modelo listos.")

🚀 Iniciando servidor Ollama...
⬇️  Descargando modelo llama3.2:1b ...

✅ Servidor y modelo listos.


**# CELDA 3 — Configuración del sistema**

¿Qué hace esta celda?
Define el diccionario config, que es el panel de control del sistema. El estudiante puede modificar:

*   urls → Los dominios que el agente consultará obligatoriamente
*   modelo → El LLM a usar (si se descargó otro con ollama pull)
*   temperatura → Controla la creatividad del modelo (0 = determinista, 1 = creativo)
*   tipo_memoria → "buffer" guarda toda la conversación; "summary" la resume (útil para conversaciones largas)

💡 **Ejercicio**: Cambia las URLs a cualquiera que sea de tu area de investigación

In [5]:
# ==============================================================================
# CELDA 3: CONFIGURACIÓN — MODIFICA AQUÍ
# ==============================================================================

config = {
    "urls": {
        "fuente_a": "fundeu.es",       # 👈 cambia por cualquier dominio
        "fuente_b": "www.rae.es"   # 👈 cambia por cualquier dominio
    },
    "modelo": "llama3.2:1b",
    "temperatura": 0.1,
    "tipo_memoria": "buffer",  # "buffer" | "ventana"
    "ventana_memoria": 6
}

print("⚙️  Configuración activa:")
for k, v in config.items():
    print(f"   {k}: {v}")

⚙️  Configuración activa:
   urls: {'fuente_a': 'fundeu.es', 'fuente_b': 'www.rae.es'}
   modelo: llama3.2:1b
   temperatura: 0.1
   tipo_memoria: buffer
   ventana_memoria: 6


# **📋 CELDA 4 — Memoria, LLM y herramientas de búsqueda**

¿Qué hace esta celda?
Instancia los tres componentes centrales del agente:

* Memoria: Almacena el historial de la conversación para que el agente recuerde el contexto. ConversationBufferMemory guarda todos los mensajes; ConversationSummaryMemory genera un resumen progresivo (más eficiente para sesiones largas).
* LLM: El cerebro del sistema. Se conecta al servidor Ollama local con el modelo y temperatura definidos en config.
* Herramientas: Cada Tool es una capacidad que el agente puede invocar. Aquí envolvemos DuckDuckGo para que filtre resultados por dominio usando site.

In [6]:
# ==============================================================================
# CELDA 4: HERRAMIENTAS, LLM E HISTORIAL
# ==============================================================================

# --- Motor de búsqueda base ---
_search = DuckDuckGoSearchRun()

def _buscar(query: str, clave: str) -> str:
    sitio = config["urls"][clave]
    resultado = _search.run(f"site:{sitio} {query}")
    return f"[Fuente: {sitio}]\n{resultado}"

# --- Herramientas ---
@tool
def buscar_fuente_a(query: str) -> str:
    """Busca noticias y artículos en la fuente A. SIEMPRE debes llamar esta herramienta primero."""
    return _buscar(query, "fuente_a")

@tool
def buscar_fuente_b(query: str) -> str:
    """Busca noticias y artículos en la fuente B. SIEMPRE debes llamar esta herramienta después de la fuente A."""
    return _buscar(query, "fuente_b")

tools = [buscar_fuente_a, buscar_fuente_b]
tools_map = {t.name: t for t in tools}

# --- LLM (sin bind_tools, no lo necesitamos) ---
llm = ChatOllama(model=config["modelo"], temperature=config["temperatura"])

# --- Historial de sesión ---
historial: list = []

def obtener_contexto() -> list:
    """Devuelve el historial completo o solo los últimos N turnos."""
    if config["tipo_memoria"] == "ventana":
        limite = config["ventana_memoria"] * 2
        return historial[-limite:] if len(historial) > limite else historial
    return historial

tipo_label = (
    "buffer completo" if config["tipo_memoria"] == "buffer"
    else f"ventana de {config['ventana_memoria']} turnos"
)

print(f"🔧 Herramientas: buscar_fuente_a, buscar_fuente_b (ejecución forzada)")
print(f"🤖 LLM: {config['modelo']} (temperatura={config['temperatura']})")
print(f"🧠 Memoria: {tipo_label}")

🔧 Herramientas: buscar_fuente_a, buscar_fuente_b (ejecución forzada)
🤖 LLM: llama3.2:1b (temperatura=0.1)
🧠 Memoria: buffer completo


# **📋 CELDA 5 — Loop del agente**

Esta celda implementa el ciclo Razonar → Actuar → Observar manualmente, sin depender de AgentExecutor:

* El LLM recibe el mensaje + historial y decide qué herramientas invocar (tool_calls)
* Se ejecutan las herramientas y sus resultados se añaden como ToolMessage
* El LLM recibe los resultados y genera la respuesta final

max_iteraciones evita bucles infinitos si el modelo sigue llamando herramientas sin llegar a una respuesta.

In [7]:
# ==============================================================================
# CELDA 5: LOOP DEL AGENTE (cadena secuencial forzada)
# ==============================================================================

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

SYSTEM_MSG = (
    "Eres un Agente experto en linguistica y reglas gramaticales. Se te entregarán resultados de búsqueda "
    "de dos fuentes diferentes la rae y fundeu. Tu tarea es:\n"
    "1. Extraer los puntos clave de cada fuente.\n"
    "2. Identificar coincidencias y diferencias entre ambas.\n"
    "3. Corregir oraciones dadas por el usuario.\n"
)

def loop_agente(mensaje_usuario: str) -> str:
    """Busca en ambas fuentes y sintetiza con el LLM."""

    # Paso 1: búsquedas forzadas (no dependemos del modelo)
    print(f"🔍 Buscando en {config['urls']['fuente_a']}...")
    resultado_a = _buscar(mensaje_usuario, "fuente_a")

    print(f"🔍 Buscando en {config['urls']['fuente_b']}...")
    resultado_b = _buscar(mensaje_usuario, "fuente_b")

    # Paso 2: construir prompt con los resultados
    prompt_sintesis = (
        f"Pregunta del usuario: {mensaje_usuario}\n\n"
        f"--- RESULTADO FUENTE A ({config['urls']['fuente_a']}) ---\n{resultado_a}\n\n"
        f"--- RESULTADO FUENTE B ({config['urls']['fuente_b']}) ---\n{resultado_b}\n\n"
        f"Ahora corrigue al usuario:"
    )

    # Paso 3: síntesis con el LLM + historial
    mensajes = (
        [SystemMessage(content=SYSTEM_MSG)]
        + obtener_contexto()
        + [HumanMessage(content=prompt_sintesis)]
    )

    respuesta = llm.invoke(mensajes)
    return respuesta.content

print("✅ Loop del agente listo.")

✅ Loop del agente listo.


# **📋 CELDA 6 — Interfaz Gradio y túnel Ngrok**

Lanza la interfaz y la expone públicamente.

Cada respuesta del agente se guarda en historial para mantener el contexto entre turnos.
Para obtener una URL pública permanente, añade tu token de ngrok.com en Secrets de Colab con el nombre NGROK_TOKEN.

In [8]:
# ==============================================================================
# CELDA 6: INTERFAZ GRADIO + TÚNEL NGROK
# ==============================================================================

def chat_handler(mensaje: str, hist: list) -> str:
    try:
        respuesta = loop_agente(mensaje)
        # Guardar turno en el historial global
        historial.append(HumanMessage(content=mensaje))
        historial.append(AIMessage(content=respuesta))
        return respuesta
    except Exception as e:
        return f"⚠️ Error: {str(e)}\n\nIntenta reformular tu pregunta."

# --- Ngrok ---
url_publica = None
try:
    token = userdata.get('NGROK_TOKEN')
    ngrok.set_auth_token(token)
    url_publica = ngrok.connect(7860)
    print(f"\n{'='*55}")
    print(f"🌍 URL PÚBLICA: {url_publica}")
    print(f"{'='*55}\n")
except Exception:
    print("⚠️  Sin NGROK_TOKEN — Gradio generará su propio enlace temporal.\n")

# --- Interfaz ---
demo = gr.ChatInterface(
    fn=chat_handler,
    title="🔍 Agente RAG Multi-Fuente",
    description=(
        f"Consulta **{config['urls']['fuente_a']}** y **{config['urls']['fuente_b']}** "
        f"y sintetiza una respuesta.\n*Modelo: `{config['modelo']}` · Memoria: {tipo_label}*"
    ),
    theme="soft",
    examples=[
        "¿Cuáles son las últimas tendencias en inteligencia artificial?",
        "¿Qué se dice sobre startups de tecnología este mes?",
        "Compara las perspectivas sobre el futuro de los LLMs"
    ]
)

demo.launch(share=not bool(url_publica))

⚠️  Sin NGROK_TOKEN — Gradio generará su propio enlace temporal.



/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5fa3123e7df41eb23a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **📋 CELDA 7 — Cierre de sesión y limpieza de recursos**

**¿Qué hace esta celda?**
Apaga ordenadamente todos los procesos activos para que no queden servicios corriendo en segundo plano ni consumiendo recursos de Colab:

* Gradio → Cierra el servidor web y libera el puerto 8000
* Ngrok → Desconecta todos los túneles públicos (la URL deja de funcionar inmediatamente)
* Ollama → Mata el proceso del servidor LLM
* Memoria → Limpia el historial de conversación de la sesión

⚠️ Ejecútala siempre al terminar la clase antes de cerrar Colab.

In [ ]:
# ==============================================================================
# CELDA 7: CIERRE DE SESIÓN Y LIMPIEZA DE RECURSOS
# ==============================================================================

import subprocess

print("🔄 Iniciando cierre ordenado del sistema...\n")

# 1. Cerrar interfaz Gradio
try:
    demo.close()
    print("✅ Servidor Gradio cerrado (puerto 8000 liberado)")
except Exception as e:
    print(f"⚠️  Gradio: {e}")

# 2. Desconectar túneles Ngrok
try:
    ngrok.disconnect(url_publica)
    ngrok.kill()
    print("✅ Túneles Ngrok desconectados (URLs públicas inactivas)")
except Exception as e:
    print(f"⚠️  Ngrok: {e}")

# 3. Apagar servidor Ollama
try:
    subprocess.run(["pkill", "-f", "ollama"], check=True)
    print("✅ Servidor Ollama detenido")
except Exception as e:
    print(f"⚠️  Ollama: {e}")

# 4. Limpiar memoria del agente
try:
    memoria.clear()
    print("✅ Historial de conversación borrado")
except Exception as e:
    print(f"⚠️  Memoria: {e}")

print("\n" + "="*50)
print("🏁 Sistema completamente apagado.")
print("   Puedes cerrar la pestaña de Colab con seguridad.")
print("="*50)

🔄 Iniciando cierre ordenado del sistema...

⚠️  Gradio: name 'demo' is not defined
⚠️  Ngrok: name 'ngrok' is not defined
⚠️  Ollama: Command '['pkill', '-f', 'ollama']' returned non-zero exit status 1.
⚠️  Memoria: name 'memoria' is not defined

🏁 Sistema completamente apagado.
   Puedes cerrar la pestaña de Colab con seguridad.
